In [1]:
from pathlib import Path
import sys
import pandas as pd

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

In [2]:
from global_package.collimators import Collimators
from global_package.bunches import Bunches
from global_package.blms import BLMs

In [75]:
# Start of the measurements
start_time = pd.to_datetime('2022-06-28 20:33:00')
# End of the measurements
end_time = pd.to_datetime('2022-06-28 20:45:30')

In [76]:
tfs_path = '/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX/levelling.20/all_optics_B4.tfs'

cols = Collimators(start_time, end_time, 'B2V', tfs_path, spark)

In [77]:
blms = BLMs(start_time, end_time, 'B2V', spark, peaks=cols.peaks.peaks)

/eos/user/m/morwat/.local/lib/python3.9/site-packages/lossmaps/nxcals.py:83: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.



In [91]:
blms.threshold = 0.9
blms.find_highest_losses()

In [92]:
blms.bottleneck

'BLMQI.03R1.B1E30_MQXA'

In [80]:
cols.reference_collimator

'TCP.D6R7.B2:MEAS_LVDT_GD'

In [93]:
def find_all_peaks_test(peaks, df_blm_col, df_blm_mqx, bottleneck, prominence, min_separation):

    # Find peaks in the normalized columns
    all_peaks1, _ = find_peaks(df_blm_col.loss / df_blm_col.loss.max(), prominence=prominence, distance=min_separation)
    all_peaks2, _ = find_peaks(df_blm_mqx[bottleneck] / df_blm_mqx[bottleneck].max(), prominence=prominence, distance=min_separation)

    # Combine `all_peaks1` and `all_peaks2`
    candidate_peaks = np.concatenate([all_peaks1, all_peaks2])
    
    # Start with all `peaks`, ensuring they're always included
    final_peaks = list(peaks.peaks.values)
    
    # Add candidate peaks if they are far enough from existing peaks
    for candidate in np.sort(candidate_peaks):
        if all(abs(candidate - existing_peak) >= min_separation for existing_peak in final_peaks):
            final_peaks.append(candidate)
    
    # Sort final peaks
    final_peaks = np.sort(final_peaks)
    
    # Get corresponding times for final peaks
    corresponding_times = df_blm_mqx['time'].iloc[final_peaks].values

    # Create a DataFrame with the results
    data = pd.DataFrame({
        'time': corresponding_times,
        'peaks': final_peaks
    })

    return data, all_peaks1, all_peaks2

In [94]:
from scipy.signal import find_peaks
import numpy as np
all_peaks, all_peaks1, all_peaks2 = find_all_peaks_test(cols.peaks, cols.ref_col_blm_df, blms.blm_mqx_df, blms.bottleneck, 0.1, 5)

In [95]:
bunches = Bunches(start_time, end_time, 'B2V', spark, cols.peaks, all_peaks)

In [96]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the first trace (Gap) to the primary y-axis
fig.add_trace(go.Scatter(x=cols.ref_col_df.time, y=cols.ref_col_df.gap, mode='lines', name='Gap'), secondary_y=False)

# Add the second trace (Loss) to the secondary y-axis
fig.add_trace(go.Scatter(x=cols.ref_col_blm_df.time, y=cols.ref_col_blm_df.loss / cols.ref_col_blm_df.loss.max(), mode='lines', name='Loss'), secondary_y=True)
fig.add_trace(go.Scatter(x=cols.ref_col_blm_df.time.iloc[cols.peaks.peaks.values], y=cols.ref_col_blm_df.loss.iloc[cols.peaks.peaks.values] / cols.ref_col_blm_df.loss.max(), mode='markers', name='Peaks'), secondary_y=True)

fig.add_trace(go.Scatter(x=bunches.bunch_df.time, y=bunches.smoothed_bunch_intensity, mode='lines', name='Smoothed Bunch Intensity', yaxis="y3"))
fig.add_trace(go.Scatter(x=bunches.bunch_df.time, y=bunches.bunch_df[bunches.bunch], mode='lines', name='Smoothed Bunch Intensity', yaxis="y3"))

# Add another trace for Loss at bottleneck (on secondary y-axis)
fig.add_trace(go.Scatter(x=blms.blm_mqx_df.time, y=blms.blm_mqx_df[blms.bottleneck] / blms.blm_mqx_df[blms.bottleneck].max(), mode='lines', name='Loss at bottleneck'), secondary_y=True)
fig.add_trace(go.Scatter(x=blms.blm_mqx_df.time, y=blms.blm_mqx_df['BLMQI.03R1.B1E30_MQXA'] / blms.blm_mqx_df['BLMQI.03R1.B1E30_MQXA'].max(), mode='lines', name='Loss at bottleneck'), secondary_y=True)

fig.add_trace(go.Scatter(x=blms.blm_mqx_df.time.iloc[all_peaks.peaks.values], y=blms.blm_mqx_df[blms.bottleneck].iloc[all_peaks.peaks.values] / blms.blm_mqx_df[blms.bottleneck].max(), mode='markers', name='All peaks'), secondary_y=True)

# Update layout for the tertiary y-axis with adjustments
fig.update_layout(
    title='Gap, Loss, and Bunch Intensity Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Gap'),  # Primary y-axis title
    yaxis2=dict(title='Loss', overlaying='y', side='right', tickfont=dict(size=10)),  # Secondary y-axis
    yaxis3=dict(  # Tertiary y-axis configuration
        title="Bunch Intensity",
        titlefont=dict(color="blue", size=10),
        tickfont=dict(color="blue", size=9),  # Smaller font for tick labels
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85  
    )
)

# Show the figure
fig.show()

In [97]:
def normalise_again(array):
    return array/array.max()

In [98]:
def normalise_blm_data(df_blm, column, peaks, protons_lost):
    
    # Convert both Series to NumPy arrays to ensure alignment
    loss_values = df_blm[column].iloc[peaks.peaks.values].values
    protons_lost_values = protons_lost.protons_lost.values 

    # Perform the normalized division
    normalised_blm = loss_values / protons_lost_values
    
    return normalised_blm

In [99]:
normalised_col_blm = normalise_blm_data(cols.ref_col_blm_df, 'loss', cols.peaks, bunches.protons_lost)
normalised_bottleneck_blm = normalise_blm_data(blms.blm_mqx_df, blms.bottleneck, cols.peaks, bunches.protons_lost)

In [100]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Create a subplot with a secondary y-axis
fig = go.Figure()

fig.add_trace(go.Scatter(x=cols.gaps, y=normalise_again(normalised_col_blm[:-1]), mode='lines+markers', name='coll'))

fig.add_trace(go.Scatter(x=cols.gaps, y=normalise_again(normalised_bottleneck_blm[:-1]), mode='lines+markers', name='bottleneck'))

#fig.add_trace(go.Scatter(x=np.delete(cols.gaps, -3), y=normalise_again(np.delete(normalised_col_blm, -3)), mode='lines+markers', name='coll'))

#fig.add_trace(go.Scatter(x=np.delete(cols.gaps, -3), y=normalise_again(np.delete(normalised_bottleneck_blm, -3)), mode='lines+markers', name='bottleneck'))

fig.update_layout(
    width=800,  # Set width in pixels
    height=600  # Set height in pixels
)

# Show the figure
fig.show()

In [89]:
class Tool():
    
    def __init__(self, 
                 collimators, 
                 blms, 
                 bunches):

IndentationError: expected an indented block (806777590.py, line 6)